In [1]:
import pandas as pd
import numpy as np

In [2]:
import xarray as xr

In [9]:
from datetime import datetime
import time
import pandas as pd

from src.constants import *
from src.data.poloniex import Poloniex

In [10]:
t = datetime(2020, 1, 20, 11, 9, 26)
t = int(time.mktime(t.timetuple()))

print(pd.to_datetime(t, unit='s'))


2020-01-20 19:09:26
2020-01-20 11:09:26


In [5]:
minute = 60
five_minutes = 5*minute

polo = Poloniex()

start = datetime(2020, 1, 20, 3, 9)
start = int(time.mktime(start.timetuple()))

end= datetime(2020, 1, 20, 3, 20)
end= int(time.mktime(end.timetuple()))
print(pd.to_datetime(end, unit='s'))
datetime.fromtimestamp(end)

2020-01-20 11:20:00


datetime.datetime(2020, 1, 20, 3, 20)

In [6]:
tick = polo.marketTicker()
tick['USDC_BTC']

{'id': 224,
 'last': '11769.16394365',
 'lowestAsk': '11775.93412391',
 'highestBid': '11772.10724296',
 'percentChange': '-0.01281966',
 'baseVolume': '1805522.06740540',
 'quoteVolume': '152.33032802',
 'isFrozen': '0',
 'high24hr': '12089.27602694',
 'low24hr': '11619.34350173'}

In [12]:
pd.to_datetime(list(range(start, end+1, 300)),unit='s')

DatetimeIndex(['2020-01-20 11:09:00', '2020-01-20 11:14:00',
               '2020-01-20 11:19:00'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
chart = polo.marketChart(period=five_minutes, start=start, end=end, pair = 'USDC_BTC')
for i in chart:
    print(i)

print(len(chart))

first = chart[0]['date']
print(pd.to_datetime(start))
print(pd.to_datetime(first))

last = chart[-1]['date']
print(pd.to_datetime(end))
print(pd.to_datetime(last))

In [16]:
data = np.random.randint(1,10, (5*2, 3))
arrays = [['red', 'blue','btc', 'eth', 'snx'], [i for i in range(2)]]
index = pd.MultiIndex.from_product(arrays)
df = pd.DataFrame(data, index = index, columns=['open', 'close', 'vol'])
print(df)
df.values.reshape(5,2,3)


        open  close  vol
red  0     5      2    2
     1     3      5    6
blue 0     2      5    8
     1     7      9    4
btc  0     6      1    2
     1     7      8    6
eth  0     8      6    4
     1     4      1    2
snx  0     2      5    4
     1     6      6    2


array([[[5, 2, 2],
        [3, 5, 6]],

       [[2, 5, 8],
        [7, 9, 4]],

       [[6, 1, 2],
        [7, 8, 6]],

       [[8, 6, 4],
        [4, 1, 2]],

       [[2, 5, 4],
        [6, 6, 2]]])

In [ ]:
import time
now = int(time.time())
time_index = pd.to_datetime(list(range(now - 300*10, now, 300)),unit='s')
time_index

In [20]:
def xarray_fillna(panel, type="bfill"):
    if type=="both":
        panel = panel.bfill(dim = 'time_index').ffill(dim = 'time_index')
    elif type=="bfill":
        panel = panel.bfill(dim = 'time_index')
    else:
        panel = panel.ffill(dim = 'time_index')
    return panel

In [3]:
data = xr.DataArray(dims=("x", "y"), coords={"x": [10, 20], "y": [i for i in range(5)]})

In [4]:
data[0,2] = 5
data

<xarray.DataArray (x: 2, y: 5)>
array([[nan, nan,  5., nan, nan],
       [nan, nan, nan, nan, nan]])
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 0 1 2 3 4

In [6]:
data.ffill('y')

<xarray.DataArray (x: 2, y: 5)>
array([[nan, nan,  5.,  5.,  5.],
       [nan, nan, nan, nan, nan]])
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 0 1 2 3 4

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class Cash(tf.keras.Model):

    def __init__(self):
        super(Cash, self).__init__()

        self.dense = tf.keras.layers.Dense(5, activation='relu')
 
        self.b = tf.Variable(tf.zeros((1, 1), dtype=tf.float32), trainable=True)
        self.out = tf.keras.layers.Activation('softmax')

    def call(self, x):
        x = self.dense(x)
        bias = tf.tile(self.b, [tf.shape(x)[0], 1])
        x = tf.concat((bias, x), axis = -1)
        x = self.out(x)
        
        return x
    
    
model = Cash()

X = np.random.randn(100, 3)
with tf.GradientTape() as tape:
    y = model(X)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, model.trainable_variables)

In [ ]:
class Cash(tf.keras.Model):

    def __init__(self, rows = 11, cols = 50, features = 3):
        super(Cash, self).__init__()
        
        input_shape = (rows, cols, features)
        inputs = tf.keras.Input(shape=input_shape)
        x = tf.keras.layers.Conv2D(
            filters = 2, 
            kernel_size = (1,3), 
            padding='valid', 
            activation='relu',
            name = 'conv1'
        )(inputs)
        x = tf.keras.layers.Conv2D(1, 
                                (1, x.shape[2]), 
                                activation="relu", 
                                name = 'conv2')(x)
        x = tf.squeeze(x)
        
        self.model = tf.keras.Model(inputs = inputs, outputs = x)
        
        self.b = tf.Variable(tf.zeros((1, 1), dtype=tf.float32), trainable=True)
        self.out = tf.keras.layers.Activation('softmax')

    def call(self, x):
        x = self.model(x)
        print(x.shape)
        bias = tf.tile(self.b, [tf.shape(x)[0], 1])
        x = tf.concat((bias, x), axis = -1)
        x = self.out(x)
        
        return x
    
    
model = Cash()

X = np.random.randn(100, 11, 50, 3)
with tf.GradientTape() as tape:
    y = model(X)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, model.trainable_variables)

In [ ]:
batch = 100
dense = tf.keras.layers.Dense(5, activation='relu')
b = tf.Variable(tf.zeros((1,1), dtype=tf.float32), name='b', trainable=True)

x =  tf.random.normal((batch, 3))
with tf.GradientTape() as tape:
    y = dense(x)
    cash = tf.tile(b, [y.shape[0], 1])
    y = tf.concat((cash, y), axis = -1)
    y = tf.keras.layers.Activation('softmax')(y)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, [b, dense.trainable_variables])
grads

In [ ]:
def CNN(rows, cols, features, batch_size):
    input_shape = (rows, cols, features)
    X = keras.Input(shape= input_shape, batch_size=batch_size)
    w = keras.Input(shape = (rows, 1, 1), batch_size=batch_size)

    x = tf.keras.layers.Conv2D(
        filters = 2, 
        kernel_size = (1,3), 
        padding='valid', 
        activation='relu'
    )(X)
        
    x = keras.layers.Conv2D(20, 
                            (1, x.shape[2]), 
                            activation="relu", 
                            name = 'conv2')(x)  
    
    con = keras.layers.Concatenate(axis=3)([x, w])

    x = keras.layers.Conv2D(1, (1,1), name = 'votes')(con)
    x = tf.squeeze(x)
    
    b = tf.tile(b, [x.shape[0], 1])
    with_bias = keras.layers.Concatenate(axis=1)([b, x])
    #with_bias = CashBias()(x)

    outputs = keras.layers.Activation('softmax')(with_bias)
    return keras.Model(inputs = [X, w], outputs = outputs, name = "Policy")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_train, (-1, 784))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
for (x, y) in train_dataset:

    with tf.GradientTape() as tape:

        # Run the forward pass of the layer.
        # The operations that the layer applies
        # to its inputs are going to be recorded
        # on the GradientTape.
        logits = model(x, training=True)  # Logits for this minibatch

        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y, logits)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    # optimizer.apply_gradients(zip(grads, model.trainable_weights))
    shapes = [grad.shape for grad in grads]
    print(shapes)
    print(model.trainable_variables)
    break

In [ ]:
a = [np.random.rand(2,3) for i in range(10)]

In [ ]:
np.array(a, dtype='float32').dtype